In [1]:
import pandas as pd
import geopandas
from opt_funcs import optimise

In [2]:
buildings_df = pd.read_csv("save_files/buildings_all_zones_df.csv")
grid_gdf = geopandas.read_file("save_files/grid_250.shp")
grid_intervals = 250

In [3]:
res = optimise(n_fac=12,
               n_iterations=10,
               buildings_df=buildings_df,
               grid_gdf=grid_gdf,
               grid_intervals=grid_intervals)

Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call
Call


In [36]:
vardict = {}
obdict = {}
for r in range(len(res.result)):
    vardict[r] = res.result[r].variables
    obdict[r] = res.result[r].objectives[:]
    
pd.DataFrame(obdict).to_csv("save_files/ob_init.csv")
pd.DataFrame(vardict).to_csv("save_files/var_init.csv")

In [35]:
pd.DataFrame(obdict)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,5.950896,6.414668,4.133209,3.433216,4.036875,4.463106,5.435522,6.517595,5.849011,4.457452,...,3.918397,4.558538,5.848685,5.101477,6.956751,4.940179,5.604505,4.355596,3.602276,4.340927
1,13.587901,13.174328,9.033095,6.701938,8.957908,10.834064,10.446656,11.246215,10.869900,8.285138,...,9.043956,12.169291,13.314219,9.522036,12.104686,9.214891,16.331888,8.773623,8.341922,7.778465
2,36000.000000,36000.000000,36000.000000,36000.000000,33000.000000,36000.000000,33000.000000,36000.000000,36000.000000,36000.000000,...,36000.000000,36000.000000,36000.000000,36000.000000,36000.000000,36000.000000,36000.000000,36000.000000,36000.000000,36000.000000
3,16969.500000,22840.500000,20248.500000,20296.500000,10108.500000,19024.500000,27537.500000,45912.500000,13185.500000,18812.500000,...,21498.500000,22920.500000,26195.500000,25230.500000,32644.500000,31428.500000,22764.500000,28652.500000,13701.500000,14271.500000


In [32]:
res.result[r].objectives[:]

[4.340926839832619, 7.778464970978855, 36000, 14271.5]